In [48]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error
from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [49]:
renda=pd.read_csv('previsao_de_renda.csv')
renda.head()

,Unnamed: 0,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,0,2015-01-01,15056,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,1,2015-01-01,9968,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,2,2015-01-01,4312,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,3,2015-01-01,10639,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,4,2015-01-01,7064,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


In [50]:
renda.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Ajustando modelo para prever a log da renda considerando todas as covariaveis disponíveis 

In [51]:
renda['qtd_filhos'].value_counts()

0     10376
1      3037
2      1376
3       185
4        17
7         5
14        2
5         2
Name: qtd_filhos, dtype: int64

In [52]:
renda['tipo_renda'].value_counts()

Assalariado         7633
Empresário          3508
Pensionista         2582
Servidor público    1268
Bolsista               9
Name: tipo_renda, dtype: int64

In [53]:
renda['educacao'].value_counts()

Secundário             8895
Superior completo      5335
Superior incompleto     579
Primário                165
Pós graduação            26
Name: educacao, dtype: int64

In [54]:
renda['estado_civil'].value_counts()

Casado      10534
Solteiro     1798
União        1078
Separado      879
Viúvo         711
Name: estado_civil, dtype: int64

In [55]:
renda['tipo_residencia'].value_counts()

Casa             13532
Com os pais        675
Governamental      452
Aluguel            194
Estúdio             83
Comunitário         64
Name: tipo_residencia, dtype: int64

In [59]:
model = smf.ols('np.log(renda) ~ data_ref + sexo + posse_de_veiculo + posse_de_imovel + C(qtd_filhos,Treatment(0)) + C(tipo_renda,Treatment(0)) + C(educacao,Treatment(2)) + C(estado_civil,Treatment(0)) + C(tipo_residencia,Treatment(1)) + idade + tempo_emprego + C(qt_pessoas_residencia)',data=renda).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.360
Model:                            OLS   Adj. R-squared:                  0.357
Method:                 Least Squares   F-statistic:                     144.9
Date:                Wed, 17 May 2023   Prob (F-statistic):               0.00
Time:                        14:41:06   Log-Likelihood:                -13545.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12378   BIC:                         2.755e+04
Df Model:                          48                                         
Covariance Type:            nonrobust                                         
                                                        coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------


* Vemos que para as variaveis **data_ref,posse_de_veiculo,tipo_renda,educacao,estado_civil,qtd_filhos,qt_pessoas_residencia, e tipo_residencia** não parecem significativa para ajudar a explicar a variavél **renda**, sendo assim, vamos remover essas variavéis. 

2. Removendo variaveis não significativas

In [57]:

model2 = smf.ols('np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + idade + tempo_emprego',data=renda).fit()
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.346
Model:                            OLS   Adj. R-squared:                  0.346
Method:                 Least Squares   F-statistic:                     1315.
Date:                Wed, 17 May 2023   Prob (F-statistic):               0.00
Time:                        14:39:54   Log-Likelihood:                -13676.
No. Observations:               12427   AIC:                         2.736e+04
Df Residuals:                   12421   BIC:                         2.741e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept               

* Aparentemente o modelo ficou pior.
2. **posse_de_veiculo** pessoas que possuem imovel ganham *0.0552*;
3. **posse_de_imovel** pessoas que possuem imovel ganham *0.0871*;
4. **idade** para cada *0.0048* log da idade maior será a renda;
5. **tempo_emprego** a cada *0.0610      * a mais de tempo de emprego mais será a renda;
6. **log qt_pessoas_residencia** para cada *0.0161* do log da quantidade de pessoas na residencia maior será a renda;


3.Comparando modelos final com inicial

*Paramêtros* | *Primeiro modelo*  |*Segundo modelo*|
:----------------|:-----------:       |-------------- :|     
$${R^2}$$            |  0.360           | 0.346
$${R^2}-ajustado$$        |  0.357           |0.346
**AIC**             |  2.718e+04       |2.736e+04

* Realizando os comparativos o segundo modelo performou pior que o primeiro modelo, mesmo após a retirada de variaveis não significativas. 